In [33]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import UpSampling2D, Cropping2D
upsample_size = 1
mask_size = np.ceil(np.array((32, 32), dtype=float) /upsample_size)
mask_size = mask_size.astype(int)
mask = np.zeros(mask_size)
pattern = np.zeros((32, 32, 3))
mask = np.expand_dims(mask, axis=2) #mask.shape=(32,32,1)
print(mask.shape)   #(32,32,1)
# 拷贝和mask和pattern一样维度的mask_tanh 和pattern_tanh
mask_tanh = np.zeros_like(mask)     
pattern_tanh = np.zeros_like(pattern)
mask_tanh_tensor = K.variable(mask_tanh)   #返回一个K变量实例，包含keras meta data
mask_tensor_unrepeat = (K.tanh(mask_tanh_tensor) \
            / (2 - K.epsilon()) + 0.5)
# 这里调用不了mask_tensor_unrepeat.eval()方法，很绝望
mask_tensor_unexpand = K.repeat_elements(
            mask_tensor_unrepeat,
            rep=3,
            axis=2)
print(mask_tensor_unrepeat.shape)   #(32,32,1)
print(mask_tensor_unexpand.shape)   #(32,32,3)



(32, 32, 1)
(32, 32, 1)
(32, 32, 3)


In [34]:
mask_tensor=K.expand_dims(mask_tensor_unexpand,axis=0)
print(mask_tensor.shape)        #(1,32,32,3)
upsample_layer = UpSampling2D(
            size=(upsample_size, upsample_size))
print(upsample_layer)
mask_upsample_tensor_uncrop = upsample_layer(mask_tensor)
uncrop_shape = K.int_shape(mask_upsample_tensor_uncrop)[1:]
print(uncrop_shape)

(1, 32, 32, 3)
(32, 32, 3)


In [35]:
cropping_layer = Cropping2D(
            cropping=((0, uncrop_shape[0] - 32),
                      (0, uncrop_shape[1] - 32)))
print('%d,%d'%(uncrop_shape[0],uncrop_shape[1]))

32,32


In [36]:
mask_upsample_tensor=cropping_layer(mask_upsample_tensor_uncrop)
pattern_tanh_tensor = K.variable(pattern_tanh)
print(pattern_tanh_tensor.shape)
pattern_raw_tensor=(K.tanh(pattern_tanh_tensor)/(2-K.epsilon()+0.5)*255.0)
print(pattern_raw_tensor.shape)

(32, 32, 3)
(32, 32, 3)


In [37]:
input_tensor = K.placeholder((None,32,32,3))
input_raw_tensor = input_tensor

print(input_tensor.shape)
print(input_raw_tensor.shape)

(?, 32, 32, 3)
(?, 32, 32, 3)
